In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [20]:
outbound_data = pd.read_excel("Outbound.xlsx")
demand_data = pd.read_excel("Demand Forecast.xlsx")
unit_data = pd.read_excel('Product Data per State.xlsx')
handling_out_data = pd.read_excel('Warehousing.xlsx')

In [21]:
# What I did: removed the State column, remove the description of tariff and small shipments
outbound_data.index = outbound_data.State
demand_data.index = demand_data.state
outbound_data = outbound_data.drop(['State','Small shipment'], axis=1)

In [22]:
# As-Is situation

def outbound_costs(dc_allocation):

    # Calculate the demand per product and DC in boxes
    dc_product_demand = {}
    for dc in dc_allocation:
        dc_product_demand[dc] = {}  # Create a dictionary for each DC
        for product in ['blender', 'swing', 'chair', 'scooter', 'skiprope']:
            total_demand_per_dc = 0
            for state in dc_allocation[dc]:
                total_demand_per_dc += (demand_data.loc[state, product])
            dc_product_demand[dc][product] = total_demand_per_dc  # Store demand (in boxes) for each product based on DC

    # Calculate the outbound costs
    shipping_costs = []
    handling_out_costs_per_dc = []
    product_units = unit_data.columns.difference(['state'])
    handling_out_data.index = handling_out_data.DC
    unit_data.index = unit_data.state

    for state in demand_data.index:
        for dc, states in dc_allocation.items():
            if state in states:
                shipping_costs.append((demand_data['total_weight'][state] * outbound_data.loc[state, dc]))

    for dc, states in dc_allocation.items():
        out_costs_per_state = 0
        for state in unit_data.index:
            if state in states:
                for product in product_units:
                    out_costs_per_state += unit_data[product][state] * handling_out_data.loc[dc, product]
                
        handling_out_costs_per_dc.append(out_costs_per_state)

    shipping_costs = sum(shipping_costs)
    handling_out_costs = sum(handling_out_costs_per_dc)
    opening_costs = len(dc_allocation)*1000000 # Keep a DC open need $1M per year.
    total_outbound_costs = np.round(shipping_costs + handling_out_costs + opening_costs, 1)

    print(dc_product_demand)
    print('Total Shipping Costs (from DC to Customers):', shipping_costs)
    print('Total Handling Out Costs:', handling_out_costs)
    print(f'Total costs for keeping {len(dc_allocation)} DCs open:', opening_costs)
    print('Total Outbound Costs:', total_outbound_costs)

    

In [23]:
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

outbound_costs(as_is_dc)

{'NY': {'blender': 226319, 'swing': 6468, 'chair': 161655, 'scooter': 3233, 'skiprope': 969935}, 'ND': {'blender': 31949, 'swing': 913, 'chair': 22820, 'scooter': 22832, 'skiprope': 136921}, 'IL': {'blender': 271812, 'swing': 7766, 'chair': 194150, 'scooter': 3884, 'skiprope': 1164904}, 'TN': {'blender': 216583, 'swing': 6188, 'chair': 154702, 'scooter': 3095, 'skiprope': 928213}, 'WA': {'blender': 51437, 'swing': 1470, 'chair': 36741, 'scooter': 735, 'skiprope': 220442}, 'TX': {'blender': 190768, 'swing': 5451, 'chair': 136263, 'scooter': 2725, 'skiprope': 817575}, 'UT': {'blender': 179502, 'swing': 5129, 'chair': 128215, 'scooter': 2564, 'skiprope': 769295}}
Total Shipping Costs (from DC to Customers): 1135481.33914202
Total Handling Out Costs: 1804220.62
Total costs for keeping 7 DCs open: 7000000
Total Outbound Costs: 9939702.0


In [26]:
# Allocate dc based on the distance/costs greedily
dc_allocation = {}
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in dc_allocation:
        dc_allocation[best_dc] = [state]
    elif isinstance(dc_allocation[best_dc], list):
        dc_allocation[best_dc].append(state)
    else:
        dc_allocation[best_dc] = [dc_allocation[best_dc], state]
        
print(outbound_costs(dc_allocation))

{'WA': {'blender': 55389, 'swing': 1583, 'chair': 39564, 'scooter': 23167, 'skiprope': 237379}, 'TN': {'blender': 295463, 'swing': 8442, 'chair': 211044, 'scooter': 4222, 'skiprope': 1266269}, 'TX': {'blender': 179449, 'swing': 5127, 'chair': 128178, 'scooter': 2563, 'skiprope': 769067}, 'UT': {'blender': 71533, 'swing': 2044, 'chair': 51094, 'scooter': 1021, 'skiprope': 306567}, 'CA': {'blender': 135932, 'swing': 3884, 'chair': 97094, 'scooter': 1942, 'skiprope': 582565}, 'NY': {'blender': 157173, 'swing': 4492, 'chair': 112265, 'scooter': 2245, 'skiprope': 673595}, 'PA': {'blender': 74153, 'swing': 2119, 'chair': 52966, 'scooter': 1060, 'skiprope': 317797}, 'KS': {'blender': 49964, 'swing': 1428, 'chair': 35688, 'scooter': 715, 'skiprope': 214129}, 'IL': {'blender': 98812, 'swing': 2823, 'chair': 70581, 'scooter': 1411, 'skiprope': 423482}, 'ND': {'blender': 50502, 'swing': 1443, 'chair': 36072, 'scooter': 722, 'skiprope': 216435}}
Total Shipping Costs (from DC to Customers): 911216.

In [28]:
greedy_dc = {}
for state in outbound_data.index:
    current_dc = outbound_data[list(as_is_dc.keys())]
    best_dc = (current_dc.loc[state]).idxmin()
    if best_dc not in as_is_dc:
        print(best_dc)
    
    if best_dc not in greedy_dc:
        greedy_dc[best_dc] = [state]
    elif isinstance(greedy_dc[best_dc], list):
        greedy_dc[best_dc].append(state)
    else:
        greedy_dc[best_dc] = [greedy_dc[best_dc], state]

outbound_costs(greedy_dc)

{'WA': {'blender': 95009, 'swing': 2715, 'chair': 67862, 'scooter': 1358, 'skiprope': 407176}, 'TN': {'blender': 343842, 'swing': 9825, 'chair': 245603, 'scooter': 4913, 'skiprope': 1473608}, 'UT': {'blender': 37800, 'swing': 1080, 'chair': 27000, 'scooter': 540, 'skiprope': 162001}, 'NY': {'blender': 341138, 'swing': 9748, 'chair': 243666, 'scooter': 4874, 'skiprope': 1462011}, 'ND': {'blender': 35015, 'swing': 1000, 'chair': 25011, 'scooter': 500, 'skiprope': 150066}, 'IL': {'blender': 259787, 'swing': 7423, 'chair': 185562, 'scooter': 3710, 'skiprope': 1113371}, 'TX': {'blender': 55779, 'swing': 1594, 'chair': 39842, 'scooter': 23173, 'skiprope': 239052}}
Total Shipping Costs (from DC to Customers): 2565522.8295609397
Total Handling Out Costs: 1830690.81
Total costs for keeping 7 DCs open: 7000000
Total Outbound Costs: 11396213.6
